# Machine Learning 1 - LSTM

## Import libraries

In [6]:
import pandas as pd
import seaborn as sns
import numpy as np
import plotly.graph_objects as go
from IPython.display import display
from sklearn.metrics import mean_squared_error as mse
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
sns.set()

In [7]:
SPX = pd.read_csv("outputs/SPX_out1.csv") 
SPX

,Date,Close,Volume,ClosePrev,CloseNext
0,1950-04-01,17.186667,2.371296e+07,NaN,18.180000
1,1950-07-01,18.180000,2.760741e+07,17.186667,18.570000
2,1950-10-01,18.570000,2.260370e+07,18.180000,19.816667
3,1951-01-01,19.816667,3.068148e+07,18.570000,21.620000
4,1951-04-01,21.620000,3.020555e+07,19.816667,21.636667
...,...,...,...,...,...
284,2021-04-01,3832.760000,5.277860e+10,3549.220000,4227.593333
285,2021-07-01,4227.593333,4.502311e+10,3832.760000,4408.493333
286,2021-10-01,4408.493333,4.207726e+10,4227.593333,4646.186667
287,2022-01-01,4646.186667,4.631885e+10,4408.493333,4433.720000


In [8]:
df_full = pd.read_csv("outputs/df_full.csv")
df_full

,Unnamed: 0,gdp,gnp,real_gdp,real_gdp_per_capita,net_exports,gni,govt_spending,consumer_spending,private_domestic_investment,cpi,consumer_oil_price,ir,unemployment_rate,Close,Volume,ClosePrev,CloseNext
0,1950-07-01,308.153,309.760,2340.112,15398.0,-0.740,307.413,600.663,200.505,1.247,24.203,11.267,1.61,4.6,18.180000,2.760741e+07,17.186667,18.570000
1,1950-10-01,319.945,321.554,2384.920,15623.0,-0.154,319.791,643.100,197.946,1.289,24.693,11.500,1.75,4.2,18.570000,2.260370e+07,18.180000,19.816667
2,1951-01-01,336.000,337.537,2417.311,15769.0,0.177,336.177,711.537,209.207,1.296,25.697,11.700,1.75,3.5,19.816667,3.068148e+07,18.570000,21.620000
3,1951-04-01,344.090,345.973,2459.196,15979.0,1.943,346.033,806.376,204.942,1.332,25.947,11.933,1.75,3.1,21.620000,3.020555e+07,19.816667,21.636667
4,1951-07-01,351.385,353.381,2509.880,16234.0,3.742,355.127,895.015,207.616,1.385,25.933,11.933,1.75,3.2,21.636667,2.172778e+07,21.620000,22.980000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,2020-04-01,19477.444,19649.442,17258.205,52031.0,-538.876,18938.568,3378.132,12989.729,519.850,256.418,219.570,0.25,13.0,2921.443333,5.985144e+10,3136.440000,3019.010000
280,2020-07-01,21138.574,21365.412,18560.774,55933.0,-725.723,20412.851,3360.238,14293.832,539.864,259.438,232.403,0.25,8.8,3019.010000,6.714349e+10,2921.443333,3378.143333
281,2020-10-01,21477.597,21728.223,18767.778,56533.0,-798.431,20679.166,3356.030,14467.611,561.269,260.879,234.862,0.25,6.8,3378.143333,5.204018e+10,3019.010000,3549.220000
282,2021-01-01,22038.226,22273.060,19055.655,57405.0,-872.540,21165.686,3390.921,15005.444,576.340,263.525,274.983,0.25,6.2,3549.220000,5.152288e+10,3378.143333,3832.760000


## Create Multi-Step LSTM Model

In [9]:
TimeSteps = 10  # next few day's Price Prediction is based on last how many past day's prices
FutureTimeSteps = 1 # How many days in future you want to predict the prices

price = SPX[['Close']].values
 
# Standardise or normalise data so it better fits into model
sc = MinMaxScaler()
DataScaler = sc.fit(price)
X = DataScaler.transform(price)
X = X.reshape(X.shape[0],)

# split into samples
X_samples = list()
y_samples = list()
NumerOfRows = len(X)
 
# Iterate thru the values to create combinations
for i in range(TimeSteps , NumerOfRows-FutureTimeSteps , 1):
    x_sample = X[i-TimeSteps:i]
    y_sample = X[i:i+FutureTimeSteps]
    X_samples.append(x_sample)
    y_samples.append(y_sample)

################################################
 
# Reshape the Input as a 3D (samples, Time Steps, Features)
X_data = np.array(X_samples)
X_data = X_data.reshape(X_data.shape[0],X_data.shape[1], 1)
# Don't reshape y as a 3D data as it is supposed to be a single column only
y_data = np.array(y_samples)


# Choosing the number of testing data records
TestingRecords = 5
# Splitting the data into train and test
X_train = X_data[:-TestingRecords]
X_test = X_data[-TestingRecords:]
y_train = y_data[:-TestingRecords]
y_test = y_data[-TestingRecords:]

# Defining Input shapes for LSTM
TimeSteps = X_train.shape[1]
TotalFeatures = X_train.shape[2]

# Initialising the RNN
regressor = Sequential()

# Adding the First input hidden layer and the LSTM layer
# return_sequences = True, means the output of every time step to be shared with hidden next layer
regressor.add(LSTM(units = 10, activation = 'relu', input_shape = (TimeSteps, TotalFeatures), return_sequences=True))


# Adding the Second hidden layer and the LSTM layer
regressor.add(LSTM(units = 5, activation = 'relu', input_shape = (TimeSteps, TotalFeatures), return_sequences=True))

# Adding the Third hidden layer and the LSTM layer
regressor.add(LSTM(units = 5, activation = 'relu', return_sequences=False ))


# Adding the output layer
# Notice the number of neurons in the dense layer is now the number of future time steps 
# Based on the number of future days we want to predict
regressor.add(Dense(units = FutureTimeSteps))

# Compiling the RNN
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

###################################################################

import time
# Measuring the time taken by the model to train
StartTime = time.time()

# Fitting the RNN to the Training set
regressor.fit(X_train, y_train, batch_size = 5, epochs = 100)

EndTime = time.time()
print("############### Total Time Taken: ", round((EndTime-StartTime)/60), 'Minutes #############')

Epoch 1/100
55/55 [==============================] - 3s 9ms/step - loss: 0.0334
Epoch 2/100
55/55 [==============================] - 0s 8ms/step - loss: 0.0186
Epoch 3/100
55/55 [==============================] - 0s 9ms/step - loss: 0.0038
Epoch 4/100
55/55 [==============================] - 0s 9ms/step - loss: 0.0020
Epoch 5/100
55/55 [==============================] - 1s 9ms/step - loss: 0.0016
Epoch 6/100
55/55 [==============================] - 1s 9ms/step - loss: 0.0013
Epoch 7/100
55/55 [==============================] - 0s 8ms/step - loss: 0.0013
Epoch 8/100
55/55 [==============================] - 0s 8ms/step - loss: 0.0012
Epoch 9/100
55/55 [==============================] - 1s 13ms/step - loss: 0.0012
Epoch 10/100
55/55 [==============================] - 0s 9ms/step - loss: 0.0011
Epoch 11/100
55/55 [==============================] - 0s 9ms/step - loss: 0.0010
Epoch 12/100
55/55 [==============================] - 0s 8ms/step - loss: 0.0011
Epoch 13/100
55/55 [================

## Test Model's Accuracy

In [10]:
df_temp = []
for i in range(10,284):
    end = i # 284 rows, so max end = 284-(no. of days predicting ahead)

    # Reshaping the data to (-1,1) because its a single entry
    LastNDaysPrices = df_full[['Close']][end-TimeSteps:end].values # [273:283]
    LastNDaysPrices = LastNDaysPrices.reshape(-1, 1)

    Original = (range(FutureTimeSteps), df_full.iloc[end:end+FutureTimeSteps, :]['Close'])[1].values

    # Scaling the data on the same level on which model was trained
    X_test = DataScaler.transform(LastNDaysPrices)

    NumberofSamples = 1
    TimeSteps = X_test.shape[0]
    NumberofFeatures = X_test.shape[1]

    # Reshaping the data as 3D input
    X_test = X_test.reshape(NumberofSamples,TimeSteps,NumberofFeatures)

    # Generating the predictions for next 5 days
    Prediction = regressor.predict(X_test)
    Prediction = DataScaler.inverse_transform(Prediction)
    Prediction = Prediction[0]
    df_temp.append(Prediction)

dfT = pd.DataFrame(df_temp)
dfT.index += 10

df_full_temp = df_full.copy()
df_full_temp = df_full_temp.reset_index()

df_new = pd.concat([df_full_temp,dfT],axis=1)
df_new = df_new.shift(-10)
df_new = df_new.dropna()
df_new.rename(columns={0:'PredictedClose'}, inplace=True)

df_new['Diff'] = df_new['Close'] - df_new['PredictedClose']
df_new['%Diff'] = ((df_new['Close'] - df_new['PredictedClose']) / df_new['Close']) * 100

In [11]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=SPX.index,
    y=SPX["Close"],
    name="Actual Price"
))


fig.add_trace(go.Scatter(
    x=df_new["index"],
    y=df_new["Diff"],
    name="Absolute Error",
    yaxis="y2"
))

fig.add_trace(go.Scatter(
    x=df_new["index"],
    y=df_new["%Diff"],
    name="Percentage Error",
    yaxis="y3"
))

fig.add_trace(go.Scatter(
    x=df_new["index"],
    y=df_new["PredictedClose"],
    name="Predicted Price",
    yaxis="y1"
))


# Create axis objects
fig.update_layout(
    xaxis=dict(
        domain=[0.3, 0.7]
    ),
    yaxis=dict(
        title="<b>Price</b>",
        titlefont=dict(
            color="#1f77b4"
        ),
        tickfont=dict(
            color="#1f77b4"
        )
    ),
    yaxis2=dict(
        title="<b>Absolute</b> Error",
        titlefont=dict(
            color="#ff7f0e"
        ),
        tickfont=dict(
            color="#ff7f0e"
        ),
        anchor="free",
        overlaying="y",
        side="left",
        position=0.15
    ),
    yaxis3=dict(
        title="<b>Percentage</b> Error (%)",
        titlefont=dict(
            color="#d62728"
        ),
        tickfont=dict(
            color="#d62728"
        ),
        anchor="x",
        overlaying="y",
        side="right"
    )
)

# Update layout properties
fig.update_layout(
    title_text='### Accuracy of the predictions: ' + str(round((100 - (100*(abs(df_new["Close"]-df_new["PredictedClose"])/df_new["Close"])).mean()),2)) + '% | Mean Square Error: ' + str(mse(df_new["Close"], df_new["PredictedClose"]).round(2)) + ' ###',
    width = 2000,
)

fig.show()

## Obtain Samples & Export

In [12]:
res_df = pd.DataFrame({'MSE':[]})

for j in range(20):
    print(f"\nTRIAL {j+1}\n")
    TimeSteps = 10  # next few day's Price Prediction is based on last how many past day's prices
    FutureTimeSteps = 1 # How many days in future you want to predict the prices

    price = SPX[['Close']].values

    # Standardise or normalise data so it better fits into model
    sc = MinMaxScaler()
    DataScaler = sc.fit(price)
    X = DataScaler.transform(price)
    X = X.reshape(X.shape[0],)

    # split into samples
    X_samples = list()
    y_samples = list()
    NumerOfRows = len(X)

    # Iterate thru the values to create combinations
    for i in range(TimeSteps , NumerOfRows-FutureTimeSteps , 1):
        x_sample = X[i-TimeSteps:i]
        y_sample = X[i:i+FutureTimeSteps]
        X_samples.append(x_sample)
        y_samples.append(y_sample)

    ################################################

    # Reshape the Input as a 3D (samples, Time Steps, Features)
    X_data = np.array(X_samples)
    X_data = X_data.reshape(X_data.shape[0],X_data.shape[1], 1)
    # Don't reshape y as a 3D data  as it is supposed to be a single column only
    y_data = np.array(y_samples)


    # Choosing the number of testing data records
    TestingRecords = 5
    # Splitting the data into train and test
    X_train = X_data[:-TestingRecords]
    X_test = X_data[-TestingRecords:]
    y_train = y_data[:-TestingRecords]
    y_test = y_data[-TestingRecords:]

    # Defining Input shapes for LSTM
    TimeSteps = X_train.shape[1]
    TotalFeatures = X_train.shape[2]

    # Initialising the RNN
    regressor = Sequential()

    # Adding the First input hidden layer and the LSTM layer
    # return_sequences = True, means the output of every time step to be shared with hidden next layer
    regressor.add(LSTM(units = 10, activation = 'relu', input_shape = (TimeSteps, TotalFeatures), return_sequences=True))


    # Adding the Second hidden layer and the LSTM layer
    regressor.add(LSTM(units = 5, activation = 'relu', input_shape = (TimeSteps, TotalFeatures), return_sequences=True))

    # Adding the Third hidden layer and the LSTM layer
    regressor.add(LSTM(units = 5, activation = 'relu', return_sequences=False ))


    # Adding the output layer
    # Notice the number of neurons in the dense layer is now the number of future time steps 
    # Based on the number of future days we want to predict
    regressor.add(Dense(units = FutureTimeSteps))

    # Compiling the RNN
    regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

    ###################################################################

    # Fitting the RNN to the Training set
    regressor.fit(X_train, y_train, batch_size = 5, epochs = 100)

    df_temp = pd.DataFrame({'Prediction':[]})
    for i in range(10,len(df_full['Close'])):
        end = i # 284 rows, so max end = 284-(no. of days predicting ahead)

        # Reshaping the data to (-1,1) because its a single entry
        LastNDaysPrices = df_full[['Close']][end-TimeSteps:end].values # [273:283]
        LastNDaysPrices = LastNDaysPrices.reshape(-1, 1)

        Original = (range(FutureTimeSteps), df_full.iloc[end:end+FutureTimeSteps, :]['Close'])[1].values

        # Scaling the data on the same level on which model was trained
        X_test = DataScaler.transform(LastNDaysPrices)

        NumberofSamples = 1
        TimeSteps = X_test.shape[0]
        NumberofFeatures = X_test.shape[1]

        # Reshaping the data as 3D input
        X_test = X_test.reshape(NumberofSamples,TimeSteps,NumberofFeatures)

        # Generating the predictions for next 5 days
        Prediction = regressor.predict(X_test)
        Prediction = DataScaler.inverse_transform(Prediction)
        new_pred = {'Prediction':Prediction[0][0]}
        df_temp = df_temp.append(new_pred, ignore_index=True)
    
    new_row = {'MSE':mse(df_full["Close"][10:], df_temp["Prediction"])}
    res_df = res_df.append(new_row, ignore_index=True)
    
    display(res_df)
    


TRIAL 1

Epoch 1/100
55/55 [==============================] - 3s 9ms/step - loss: 0.0379
Epoch 2/100
55/55 [==============================] - 0s 8ms/step - loss: 0.0237
Epoch 3/100
55/55 [==============================] - 0s 8ms/step - loss: 0.0112
Epoch 4/100
55/55 [==============================] - 1s 10ms/step - loss: 0.0026
Epoch 5/100
55/55 [==============================] - 0s 8ms/step - loss: 0.0020
Epoch 6/100
55/55 [==============================] - 0s 8ms/step - loss: 0.0018
Epoch 7/100
55/55 [==============================] - 1s 10ms/step - loss: 0.0017
Epoch 8/100
55/55 [==============================] - 0s 8ms/step - loss: 0.0015
Epoch 9/100
55/55 [==============================] - 0s 8ms/step - loss: 0.0014
Epoch 10/100
55/55 [==============================] - 0s 8ms/step - loss: 0.0015
Epoch 11/100
55/55 [==============================] - 0s 7ms/step - loss: 0.0014
Epoch 12/100
55/55 [==============================] - 0s 7ms/step - loss: 0.0014
Epoch 13/100
55/55 [=====

,MSE
0,13784.256021



TRIAL 2

Epoch 1/100
55/55 [==============================] - 3s 9ms/step - loss: 0.0345
Epoch 2/100
55/55 [==============================] - 0s 8ms/step - loss: 0.0185
Epoch 3/100
55/55 [==============================] - 0s 8ms/step - loss: 0.0069
Epoch 4/100
55/55 [==============================] - 0s 7ms/step - loss: 0.0046
Epoch 5/100
55/55 [==============================] - 0s 9ms/step - loss: 0.0040
Epoch 6/100
55/55 [==============================] - 0s 8ms/step - loss: 0.0023
Epoch 7/100
55/55 [==============================] - 0s 8ms/step - loss: 0.0015
Epoch 8/100
55/55 [==============================] - 0s 8ms/step - loss: 0.0014
Epoch 9/100
55/55 [==============================] - 0s 8ms/step - loss: 0.0013
Epoch 10/100
55/55 [==============================] - 0s 7ms/step - loss: 0.0013
Epoch 11/100
55/55 [==============================] - 0s 7ms/step - loss: 0.0012
Epoch 12/100
55/55 [==============================] - 0s 7ms/step - loss: 0.0012
Epoch 13/100
55/55 [=======

,MSE
0,13784.256021
1,23267.593622



TRIAL 3

Epoch 1/100
55/55 [==============================] - 4s 8ms/step - loss: 0.0327
Epoch 2/100
55/55 [==============================] - 1s 10ms/step - loss: 0.0190
Epoch 3/100
55/55 [==============================] - 1s 10ms/step - loss: 0.0035
Epoch 4/100
55/55 [==============================] - 0s 9ms/step - loss: 0.0015
Epoch 5/100
55/55 [==============================] - 0s 9ms/step - loss: 0.0014
Epoch 6/100
55/55 [==============================] - 0s 9ms/step - loss: 0.0012
Epoch 7/100
55/55 [==============================] - 0s 8ms/step - loss: 0.0013
Epoch 8/100
55/55 [==============================] - 0s 7ms/step - loss: 0.0011
Epoch 9/100
55/55 [==============================] - 0s 7ms/step - loss: 0.0012
Epoch 10/100
55/55 [==============================] - 0s 8ms/step - loss: 0.0012
Epoch 11/100
55/55 [==============================] - 0s 8ms/step - loss: 0.0014
Epoch 12/100
55/55 [==============================] - 0s 8ms/step - loss: 0.0012
Epoch 13/100
55/55 [=====

...

,MSE
0,13784.256021
1,23267.593622
2,15787.038724
3,10290.518741
4,13896.817178
5,663816.384904
6,9242.089766
7,11858.908939
8,12752.546323
9,13776.037066



TRIAL 19

Epoch 1/100
55/55 [==============================] - 3s 9ms/step - loss: 0.0319
Epoch 2/100
55/55 [==============================] - 0s 9ms/step - loss: 0.0202
Epoch 3/100
55/55 [==============================] - 1s 13ms/step - loss: 0.0051
Epoch 4/100
55/55 [==============================] - 1s 9ms/step - loss: 0.0016
Epoch 5/100
55/55 [==============================] - 1s 11ms/step - loss: 0.0012
Epoch 6/100
55/55 [==============================] - 1s 11ms/step - loss: 0.0012
Epoch 7/100
55/55 [==============================] - 1s 10ms/step - loss: 0.0013
Epoch 8/100
55/55 [==============================] - 1s 11ms/step - loss: 0.0013
Epoch 9/100
55/55 [==============================] - 1s 11ms/step - loss: 0.0012
Epoch 10/100
55/55 [==============================] - 1s 9ms/step - loss: 0.0012
Epoch 11/100
55/55 [==============================] - 1s 12ms/step - loss: 0.0010
Epoch 12/100
55/55 [==============================] - 1s 10ms/step - loss: 0.0010
Epoch 13/100
55/55

,MSE
0,13784.256021
1,23267.593622
2,15787.038724
3,10290.518741
4,13896.817178
5,663816.384904
6,9242.089766
7,11858.908939
8,12752.546323
9,13776.037066



TRIAL 20

Epoch 1/100
55/55 [==============================] - 3s 6ms/step - loss: 0.0346
Epoch 2/100
55/55 [==============================] - 0s 7ms/step - loss: 0.0134
Epoch 3/100
55/55 [==============================] - 0s 7ms/step - loss: 0.0030
Epoch 4/100
55/55 [==============================] - 0s 7ms/step - loss: 0.0016
Epoch 5/100
55/55 [==============================] - 0s 7ms/step - loss: 0.0014
Epoch 6/100
55/55 [==============================] - 0s 8ms/step - loss: 0.0011
Epoch 7/100
55/55 [==============================] - 1s 9ms/step - loss: 0.0011
Epoch 8/100
55/55 [==============================] - 0s 7ms/step - loss: 0.0013
Epoch 9/100
55/55 [==============================] - 0s 7ms/step - loss: 0.0010
Epoch 10/100
55/55 [==============================] - 0s 9ms/step - loss: 9.8207e-04
Epoch 11/100
55/55 [==============================] - 0s 8ms/step - loss: 9.5422e-04
Epoch 12/100
55/55 [==============================] - 0s 7ms/step - loss: 0.0011
Epoch 13/100
55/55

,MSE
0,13784.256021
1,23267.593622
2,15787.038724
3,10290.518741
4,13896.817178
5,663816.384904
6,9242.089766
7,11858.908939
8,12752.546323
9,13776.037066


## Export File

In [15]:
res_df.to_csv("outputs/res_df.csv")